In [1]:
!cd /kaggle/working/

In [2]:
!git clone https://github.com/Trusted-AI/adversarial-robustness-toolbox.git
!cd adversarial-robustness-toolbox
!pip install -e 

Cloning into 'adversarial-robustness-toolbox'...
remote: Enumerating objects: 75424, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 75424 (delta 2), reused 0 (delta 0), pack-reused 75418
Receiving objects: 100% (75424/75424), 408.48 MiB | 14.77 MiB/s, done.
Resolving deltas: 100% (59762/59762), done.

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-e option requires 1 argument


In [3]:
!cd /kaggle/working/

In [4]:
!git clone -b deepspeech https://github.com/hammaad2002/ARTToolbox.git

Cloning into 'ARTToolbox'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 362 (delta 34), reused 27 (delta 11), pack-reused 301
Receiving objects: 100% (362/362), 1.46 MiB | 7.75 MiB/s, done.
Resolving deltas: 100% (224/224), done.


In [5]:
my_string = '/kaggle/working/'

In [6]:
import os,shutil
os.remove(my_string + "adversarial-robustness-toolbox/art/attacks/evasion/imperceptible_asr/imperceptible_asr_pytorch.py")
shutil.copy(my_string + 'ARTToolbox/deepspeech/imperceptible_asr_pytorch.py', my_string + 'adversarial-robustness-toolbox/art/attacks/evasion/imperceptible_asr/')
os.remove(my_string + "adversarial-robustness-toolbox/art/estimators/speech_recognition/pytorch_deep_speech.py")
shutil.copy(my_string + 'ARTToolbox/deepspeech/pytorch_deep_speech.py',my_string + 'adversarial-robustness-toolbox/art/estimators/speech_recognition/')

'/kaggle/working/adversarial-robustness-toolbox/art/estimators/speech_recognition/pytorch_deep_speech.py'

In [7]:
%%capture
!git clone https://github.com/SeanNaren/deepspeech.pytorch.git && cd deepspeech.pytorch && python setup.py install

In [8]:
if my_string == '/kaggle/working/': #useless conditional statement
    os.chdir('/kaggle/working/deepspeech.pytorch/')
else:
    os.chdir('/content/deepspeech.pytorch/')

In [9]:
!cd /kaggle/working/deepspeech.pytorch

In [10]:
import sys
sys.path.append(my_string + "deepspeech.pytorch")

In [11]:
%%capture
!pip install pytorch-lightning

In [12]:
%%capture
!pip install omegaconf

In [13]:
%%capture
!pip install python-levenshtein

In [14]:
%%capture
!pip install hydra-core --upgrade

In [15]:
sys.path.append(my_string + "adversarial-robustness-toolbox")

In [16]:
%%capture
!git clone --recursive https://github.com/StickCui/warp-ctc-pytorch.git

In [17]:
%%capture
!pip install omegaconf --upgrade

In [18]:
%%capture
!pip install git+https://github.com/pytorch/hydra-torch

In [19]:
%%capture
!pip install hydra-configs-pytorch-lightning

In [20]:
!cd /kaggle/working/

In [21]:
%%capture
!git clone -b main https://github.com/hammaad2002/CRDNN_Model.git

In [22]:
%%capture
!pip install sox #need to install this properly for ART toolbox

In [23]:
import IPython.display as ipd

# **------ATTACK IMPLEMENTATION ON DEEPSPEECH MODEL USING ART TOOLBOX------**

# **----------------------ATTACK PREPROCESSING AND INITIALIZATION----------------------**

In [24]:
import torch
from art.estimators.speech_recognition import PyTorchDeepSpeech
from art.attacks.evasion import FastGradientMethod , BasicIterativeMethod, ProjectedGradientDescent, ImperceptibleASRPyTorch, CarliniWagnerASR
import numpy as np
import librosa

In [25]:
# Create an ART estimator for the ASR model
estimator = PyTorchDeepSpeech( pretrained_model = "librispeech" )

0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/migration/migration.py:200: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  category=PossibleUserWarning,


In [26]:
# Load some audio data for testing
x, sr = librosa.load( my_string + "/deepspeech.pytorch/" + "CRDNN_Model/AudioSamplesASR/spk1_snt1.wav" , sr=None )
x = torch.from_numpy( x )
x = x.unsqueeze( 0 ) 
x = x.numpy()
print( x.shape )

(1, 45920)


# **TARGETTED TRANSCRIPTION** 

In [27]:
y1 =  np.array(['THE MAN ALMOST KILL THE BIG CAT'])
print(y1.shape)

(1,)


# **------------------------------FAST GRADIENT DESCENT ATTACK-----------------------------**

# **PARAMETERS:**
estimator – A trained classifier.

norm – The norm of the adversarial perturbation. Possible values: “inf”, np.inf, 1 or 2.

eps – Attack step size (input variation).

eps_step – Step size of input variation for minimal perturbation computation.

targeted (bool) – Indicates whether the attack is targeted (True) or untargeted (False)

num_random_init (int) – Number of random initialisations within the epsilon ball. For random_init=0 starting at the original input.

batch_size (int) – Size of the batch on which adversarial samples are generated.

minimal (bool) – Indicates if computing the minimal perturbation (True). If True, also define eps_step for the step size and eps for the maximum perturbation.

summary_writer – Activate summary writer for TensorBoard. Default is False and deactivated summary writer. If True save runs/CURRENT_DATETIME_HOSTNAME in current directory. If of type str save in path. If of type SummaryWriter apply provided custom summary writer. Use hierarchical folder structure to compare between runs easily. e.g. pass in ‘runs/exp1’, ‘runs/exp2’, etc. for each new experiment to compare across them.

In [28]:
# Creating adversarial attack (FGSM) instance
attack1 = FastGradientMethod(estimator=estimator, batch_size= 1, eps = 0.5, targeted= True, norm = 2, eps_step = 10, num_random_init = 0)
# Generate an adversarial example from the original audio data
x_adv1 = attack1.generate(x=x, y=y1)

/bin/sh: 1: sox: not found


In [59]:
# Compare the transcription of the original and adversarial audio data
y_pred1 = estimator.predict( x )
y_adv_pred1 = estimator.predict( x_adv1 )
print("Original transcription:    ", str(y_pred1)[2:-2] )
print("Adversarial transcription: ", str(y_adv_pred1)[2:-2] )
print("Targetted Transcription:   ", y1[0] )

Original transcription:     CHILD ALMOST HURT THE SMALL DOG
Adversarial transcription:  CHILD ALMOST FHREE I SNA OF
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [30]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [31]:
ipd.Audio(x_adv1, rate = sr)

# **-----------------------------BASIC ITERATIVE METHOD ATTACK-----------------------------**

# **PARAMETERS:**
estimator – A trained classifier.

eps – Maximum perturbation that the attacker can introduce.

eps_step – Attack step size (input variation) at each iteration.

max_iter (int) – The maximum number of iterations.

targeted (bool) – Indicates whether the attack is targeted (True) or untargeted (False).

batch_size (int) – Size of the batch on which adversarial samples are generated.

verbose (bool) – Show progress bars.

In [32]:
# Creating adversarial attack (BIM) instance
attack2 = BasicIterativeMethod(estimator=estimator, batch_size= 1, eps = 0.0002, targeted= True, eps_step= 0.00001, max_iter= 100, verbose= True)
# Generate an adversarial example from the original audio data
x_adv2 = attack2.generate(x=x, y=y1)

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [60]:
# Compare the transcription of the original and adversarial audio data
y_pred2 = estimator.predict( x )
y_adv_pred2 = estimator.predict( x_adv2 )
print("Original transcription:    ", str(y_pred2)[2:-2] )
print("Adversarial transcription: ", str(y_adv_pred2)[2:-2] )
print("Targetted Transcription:   ", y1[0] )

Original transcription:     CHILD ALMOST HURT THE SMALL DOG
Adversarial transcription:  CHILD ALMOST HURT THE SMALL DARK
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [34]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [35]:
ipd.Audio(x_adv2, rate = sr)

# **--------------------------PROJECTED GRADIENT DESCENT ATTACK-------------------------**

# **PARAMETERS:**
estimator – An trained estimator.

norm – The norm of the adversarial perturbation supporting “inf”, np.inf, 1 or 2.

eps – Maximum perturbation that the attacker can introduce.

eps_step – Attack step size (input variation) at each iteration.

random_eps (bool) – When True, epsilon is drawn randomly from truncated normal distribution. The literature suggests this for FGSM based training to generalize across different epsilons. eps_step is modified to preserve the ratio of eps / eps_step. The effectiveness of this method with PGD is untested (https://arxiv.org/pdf/1611.01236.pdf).

decay – Decay factor for accumulating the velocity vector when using momentum.

max_iter (int) – The maximum number of iterations.

targeted (bool) – Indicates whether the attack is targeted (True) or untargeted (False).

num_random_init (int) – Number of random initialisations within the epsilon ball. For num_random_init=0 starting at the original input.

batch_size (int) – Size of the batch on which adversarial samples are generated.

summary_writer – Activate summary writer for TensorBoard. Default is False and deactivated summary writer. If True save runs/CURRENT_DATETIME_HOSTNAME in current directory. If of type str save in path. If of type SummaryWriter apply provided custom summary writer. Use hierarchical folder structure to compare between runs easily. e.g. pass in ‘runs/exp1’, ‘runs/exp2’, etc. for each new experiment to compare across them.

verbose (bool) – Show progress bars.

BTW THERE IS ALSO ANOTHER IMPLEMENTATION OF PGD GIVEN IN DOCUMENTATION AS : classart.attacks.evasion.ProjectedGradientDescent

AND THE DOCUMENTATION CAN BE ACCESSED THROUGH THIS LINK: https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/attacks/evasion.html#projected-gradient-descent-pgd-pytorch

In [36]:
# Creating adversarial attack (PGD) instance
attack3 = ProjectedGradientDescent(estimator=estimator, batch_size= 1, eps = 0.5, targeted= True, eps_step= 0.0055, max_iter= 200, verbose = True, norm = 2)
# Generate an adversarial example from the original audio data
x_adv3 = attack3.generate(x=x, y=y1)

PGD - Iterations:   0%|          | 0/200 [00:00<?, ?it/s]

In [56]:
# Compare the transcription of the original and adversarial audio data
y_pred3 = estimator.predict( x )
y_adv_pred3 = estimator.predict( x_adv3 )
print("Original transcription:    ", str(y_pred3)[2:-2] )
print("Adversarial transcription: ", str(y_adv_pred3)[2:-2] )
print("Targetted Transcription:   ", y1[0] )

Original transcription:     CHILD ALMOST HURT THE SMALL DOG
Adversarial transcription:  CILD ALMIST P IP THIS NON DIDE
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [38]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [39]:
ipd.Audio(x_adv3, rate = sr)

# **--------------------------------CARLINI AND WAGNER ATTACK-------------------------------**

# **PARAMETERS:**
estimator – A trained speech recognition estimator.

eps (float) – Initial max norm bound for adversarial perturbation.

learning_rate (float) – Learning rate of attack.

max_iter (int) – Number of iterations.

decrease_factor_eps (float) – Decrease factor for epsilon (Paper default: 0.8).

num_iter_decrease_eps (int) – Iterations after which to decrease epsilon if attack succeeds (Paper default: 10).

batch_size (int) – Batch size.

In [40]:
# Creating adversarial attack (CW) instance
attack4 = CarliniWagnerASR(estimator=estimator, batch_size= 1, eps = 0.0035, max_iter = 1000, learning_rate = 50.0, decrease_factor_eps= 1.0, num_iter_decrease_eps= 1)
# Generate an adversarial example from the original audio data
x_adv4 = attack4.generate(x=x, y=y1)

In [57]:
# Compare the transcription of the original and adversarial audio data
y_pred4 = estimator.predict( x )
y_adv_pred4 = estimator.predict( x_adv4 )
print("Original transcription:    ", str(y_pred4)[2:-2] )
print("Adversarial transcription: ", str(y_adv_pred4)[2:-2] )
print("Targetted Transcription:   ", y1[0] )

Original transcription:     CHILD ALMOST HURT THE SMALL DOG
Adversarial transcription:  CHILY ALMOST HERE TLE NO GON
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [42]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [43]:
ipd.Audio(x_adv4, rate = sr)

# **---------------------------------IMPERCEPTIBLE ASR ATTACK--------------------------------**

# **PARAMETERS:**
estimator – A trained speech recognition estimator.

masker – A Psychoacoustic masker.

eps (float) – Initial max norm bound for adversarial perturbation.

learning_rate_1 (float) – Learning rate for stage 1 of attack.

max_iter_1 (int) – Number of iterations for stage 1 of attack.

alpha (float) – Initial alpha value for balancing stage 2 loss.

learning_rate_2 (float) – Learning rate for stage 2 of attack.

max_iter_2 (int) – Number of iterations for stage 2 of attack.

loss_theta_min (float) – If imperceptible loss reaches minimum, stop early. Works best with batch_size=1.

decrease_factor_eps (float) – Decrease factor for epsilon (Paper default: 0.8).

num_iter_decrease_eps (int) – Iterations after which to decrease epsilon if attack succeeds (Paper default: 10).

increase_factor_alpha (float) – Increase factor for alpha (Paper default: 1.2).

num_iter_increase_alpha (int) – Iterations after which to increase alpha if attack succeeds (Paper default: 20).

decrease_factor_alpha (float) – Decrease factor for alpha (Paper default: 0.8).

num_iter_decrease_alpha (int) – Iterations after which to decrease alpha if attack fails (Paper default: 50).

batch_size (int) – Batch size.

In [44]:
# Creating adversarial attack (IMPERCEPTIBLE ASR attack) instance
attack5 = ImperceptibleASRPyTorch(estimator=estimator, batch_size= 1, eps = 0.5)
# Generate an adversarial example from the original audio data
x_adv5 = attack5.generate(x=x, y=y1)

/opt/conda/lib/python3.7/site-packages/torch/functional.py:633: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/SpectralOps.cpp:801.)
  normalized, onesided, return_complex)


In [58]:
# Compare the transcription of the original and adversarial audio data
y_pred5 = estimator.predict( x )
y_adv_pred5 = estimator.predict( x_adv5 )
print("Original transcription:    ", str(y_pred5)[2:-2] )
print("Adversarial transcription: ", str(y_adv_pred5)[2:-2] )
print("Targetted Transcription:   ", y1[0] )

Original transcription:     CHILD ALMOST HURT THE SMALL DOG
Adversarial transcription:  THE MAN ALMOST KILL THE SBWIG CAT
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [46]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [47]:
ipd.Audio(x_adv5, rate = sr)

# **----------------------------------------------------------------------------------------------------**